# 📊 Lineare Optimierung in der Produktionsplanung

**Intelligente Produktionsplanung und KI**  
Universität Stuttgart  
Master Maschinenbau / Technologiemanagement

---

## Lernziele

Nach Bearbeitung dieses Notebooks können Sie:

1. **Produktionsplanungsprobleme** als lineare Optimierungsprobleme formulieren
2. Die **Standardform** eines LP-Problems aufstellen (Zielfunktion, Nebenbedingungen)
3. LP-Probleme mit **Python (SciPy)** lösen
4. Die **graphische Lösung** für 2D-Probleme interpretieren
5. Eine **Sensitivitätsanalyse** durchführen und Schattenpreise interpretieren

---

## 1. Einführung und Grundlagen

### Was ist Lineare Optimierung?

Die **lineare Optimierung** (auch lineare Programmierung, LP) ist eine mathematische Methode zur Bestimmung des Optimums (Maximum oder Minimum) einer linearen Zielfunktion unter Berücksichtigung linearer Nebenbedingungen.

### Allgemeine Form eines LP-Problems

$$\text{Maximiere/Minimiere: } Z = c_1 x_1 + c_2 x_2 + ... + c_n x_n$$

$$\text{unter den Nebenbedingungen: } \sum_{j=1}^{n} a_{ij} x_j \leq b_i \quad \text{für } i = 1, ..., m$$

$$x_j \geq 0 \quad \text{für } j = 1, ..., n$$

### Anwendungen in der Produktionsplanung

- **Produktionsprogrammplanung**: Welche Produkte in welcher Menge fertigen?
- **Ressourcenallokation**: Wie knappe Ressourcen optimal einsetzen?
- **Mischungsprobleme**: Optimale Zusammensetzung von Produkten
- **Transportprobleme**: Kostenminimale Warenverteilung

In [ ]:
# ===================================================================
# Imports und Konfiguration
# ===================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import warnings
warnings.filterwarnings('ignore')

# Matplotlib Konfiguration für deutsche Beschriftungen
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 11

print("✅ Alle Bibliotheken erfolgreich geladen!")
print("\n📦 Verwendete Pakete:")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")

---

## 2. Fallbeispiel: Möbelhersteller "WoodCraft GmbH"

### Problemstellung

Die **WoodCraft GmbH** produziert drei Produkte: **Tische**, **Stühle** und **Schränke**. 

Für die Produktion werden drei Ressourcen benötigt:
- Holz (gemessen in m²)
- Arbeitszeit in der Schreinerei (in Stunden)
- Zeit in der Lackiererei (in Stunden)

**Ziel**: Maximierung des wöchentlichen Deckungsbeitrags bei gegebenen Kapazitätsbeschränkungen.

In [ ]:
# ===================================================================
# Problemdaten definieren
# ===================================================================

# Produktdaten als Dictionary
# HINWEIS: Die Werte sind so gewählt, dass eine interessante Mischproduktion optimal ist
produkte_info = {
    'Tisch': {
        'Deckungsbeitrag_EUR': 80, 
        'Holz_m2': 4.0, 
        'Arbeitszeit_h': 2.0, 
        'Lackierung_h': 3.0
    },
    'Stuhl': {
        'Deckungsbeitrag_EUR': 40, 
        'Holz_m2': 1.0, 
        'Arbeitszeit_h': 1.5, 
        'Lackierung_h': 1.0
    },
    'Schrank': {
        'Deckungsbeitrag_EUR': 90, 
        'Holz_m2': 3.0, 
        'Arbeitszeit_h': 3.0, 
        'Lackierung_h': 2.0
    }
}

# Kapazitätsbeschränkungen (pro Woche)
kapazitaeten = {
    'Holz_m2': 310,         # m² Holz verfügbar
    'Arbeitszeit_h': 260,   # Stunden Arbeitszeit in der Schreinerei
    'Lackierung_h': 250     # Stunden in der Lackiererei
}

# Daten als DataFrame anzeigen
print("📋 PRODUKTDATEN")
print("=" * 65)
df_produkte = pd.DataFrame(produkte_info).T
df_produkte.index.name = 'Produkt'
print(df_produkte.to_string())

print("\n📦 VERFÜGBARE KAPAZITÄTEN (pro Woche)")
print("=" * 65)
for ressource, kap in kapazitaeten.items():
    einheit = "m²" if "m2" in ressource else "Stunden"
    name = ressource.replace('_', ' ').replace('m2', '').replace('h', '')
    print(f"   {name}: {kap} {einheit}")

# Berechne Deckungsbeitrag pro Ressourceneinheit (für Verständnis)
print("\n📊 DECKUNGSBEITRAG PRO RESSOURCENEINHEIT (€)")
print("=" * 65)
print(f"{'Produkt':<10} {'DB/Holz':<12} {'DB/Arbeit':<12} {'DB/Lackierung':<12}")
print("-" * 50)
for prod, info in produkte_info.items():
    db = info['Deckungsbeitrag_EUR']
    db_holz = db / info['Holz_m2']
    db_arbeit = db / info['Arbeitszeit_h']
    db_lack = db / info['Lackierung_h']
    print(f"{prod:<10} {db_holz:<12.2f} {db_arbeit:<12.2f} {db_lack:<12.2f}")

print("\n💡 Die unterschiedlichen Verhältnisse zeigen, dass je nach Engpass")
print("   verschiedene Produkte vorteilhaft sein können!")

---

## 3. Mathematische Modellformulierung

### Schritt 1: Entscheidungsvariablen definieren

| Variable | Bedeutung |
|----------|----------|
| $x_1$ | Anzahl produzierter Tische pro Woche |
| $x_2$ | Anzahl produzierter Stühle pro Woche |
| $x_3$ | Anzahl produzierter Schränke pro Woche |

### Schritt 2: Zielfunktion formulieren

**Maximiere den Deckungsbeitrag:**

$$Z = 80 \cdot x_1 + 40 \cdot x_2 + 90 \cdot x_3 \quad [\text{EUR}]$$

### Schritt 3: Nebenbedingungen aufstellen

| Beschränkung | Formel | Kapazität |
|--------------|--------|----------|
| Holz | $4.0 x_1 + 1.0 x_2 + 3.0 x_3 \leq 310$ | m² |
| Arbeitszeit | $2.0 x_1 + 1.5 x_2 + 3.0 x_3 \leq 260$ | Stunden |
| Lackierung | $3.0 x_1 + 1.0 x_2 + 2.0 x_3 \leq 250$ | Stunden |
| Nichtnegativität | $x_1, x_2, x_3 \geq 0$ | - |

In [ ]:
# ===================================================================
# Mathematisches Modell in Matrixform
# ===================================================================

print("🔢 MATHEMATISCHES MODELL IN MATRIXFORM")
print("=" * 60)

# Koeffizientenvektor der Zielfunktion
c = np.array([80, 40, 90])
print("\nZielfunktionsvektor c (Deckungsbeiträge):")
print(f"   c = {c}")

# Koeffizientenmatrix der Nebenbedingungen
A = np.array([
    [4.0, 1.0, 3.0],  # Holz
    [2.0, 1.5, 3.0],  # Arbeitszeit
    [3.0, 1.0, 2.0]   # Lackierung
])
print("\nNebenbedingungsmatrix A:")
print(f"   {A[0]}  (Holz)")
print(f"   {A[1]}  (Arbeitszeit)")
print(f"   {A[2]}  (Lackierung)")

# Rechte Seite (Kapazitäten)
b = np.array([310, 260, 250])
print("\nKapazitätsvektor b:")
print(f"   b = {b}")

print("\n" + "-" * 60)
print("Kompakte Darstellung:")
print("   max  Z = c^T · x")
print("   s.t. A · x ≤ b")
print("        x ≥ 0")

---

## 4. Lösung mit SciPy

### Die `linprog`-Funktion

SciPy's `linprog` ist ein **Minimierungsalgorithmus**. Um ein Maximierungsproblem zu lösen, multiplizieren wir die Zielfunktion mit -1:

$$\max Z = c^T x \quad \Leftrightarrow \quad \min (-Z) = (-c)^T x$$

### Syntax

```python
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')
```

| Parameter | Bedeutung |
|-----------|----------|
| `c` | Koeffizienten der Zielfunktion (negativ bei Maximierung) |
| `A_ub` | Matrix der Ungleichungsnebenbedingungen (nur ≤ möglich!) |
| `b_ub` | Rechte Seite der Ungleichungen |
| `bounds` | Unter- und Obergrenzen für Variablen (Tupel pro Variable) |
| `method` | Lösungsalgorithmus ('highs' empfohlen) |

In [ ]:
# ===================================================================
# Lösung des Optimierungsproblems
# ===================================================================

def solve_produktionsprogramm(c, A, b, verbose=True):
    """
    Löst das Produktionsprogramm-Optimierungsproblem.
    
    Parameter:
    ----------
    c : array-like
        Deckungsbeiträge (positiv für Maximierung)
    A : array-like
        Nebenbedingungsmatrix
    b : array-like
        Kapazitätsvektor
    verbose : bool
        Ob detaillierte Ausgabe erfolgen soll
        
    Returns:
    --------
    dict : Ergebnisse der Optimierung
    """
    
    # Zielfunktion negieren für Maximierung
    c_neg = -np.array(c)
    
    # Grenzen für Variablen (alle >= 0, keine Obergrenze)
    n_vars = len(c)
    bounds = [(0, None) for _ in range(n_vars)]
    
    # Optimierung durchführen
    result = linprog(
        c_neg, 
        A_ub=A, 
        b_ub=b, 
        bounds=bounds, 
        method='highs'
    )
    
    if verbose and result.success:
        print("🎯 OPTIMALE LÖSUNG GEFUNDEN")
        print("=" * 50)
    
    return {
        'success': result.success,
        'x': result.x if result.success else None,
        'z': -result.fun if result.success else None,
        'message': result.message
    }

# Optimierung durchführen
ergebnis = solve_produktionsprogramm(c, A, b)

if ergebnis['success']:
    x_opt = ergebnis['x']
    z_opt = ergebnis['z']
    
    produkte = ['Tische', 'Stühle', 'Schränke']
    
    print("\n📊 Optimale Produktionsmengen:")
    print("-" * 40)
    for i, (produkt, menge) in enumerate(zip(produkte, x_opt)):
        db = c[i]
        gesamt_db = menge * db
        print(f"   {produkt:12s}: {menge:7.2f} Stück  →  DB: {gesamt_db:8.2f} €")
    
    print("-" * 40)
    print(f"   {'GESAMT':12s}:                  DB: {z_opt:8.2f} €")
    
    print("\n💡 INTERPRETATION:")
    print(f"   Die optimale Lösung ist eine MISCHPRODUKTION aus den Produkten.")
    print(f"   Dies liegt an den unterschiedlichen Ressourcenverbräuchen und")
    print(f"   Deckungsbeiträgen pro Ressourceneinheit.")
else:
    print(f"❌ Optimierung fehlgeschlagen: {ergebnis['message']}")

In [ ]:
# ===================================================================
# Analyse der Kapazitätsauslastung
# ===================================================================

def berechne_auslastung(x, A, b, ressourcen_namen):
    """
    Berechnet die Kapazitätsauslastung für die optimale Lösung.
    
    Returns:
    --------
    DataFrame mit Auslastungsinformationen
    """
    verbrauch = A @ x  # Matrixmultiplikation
    auslastung_prozent = (verbrauch / b) * 100
    schlupf = b - verbrauch
    
    df_auslastung = pd.DataFrame({
        'Ressource': ressourcen_namen,
        'Verbrauch': np.round(verbrauch, 2),
        'Kapazität': b,
        'Schlupf': np.round(schlupf, 2),
        'Auslastung (%)': np.round(auslastung_prozent, 1)
    })
    
    return df_auslastung

# Auslastung berechnen
ressourcen = ['Holz (m²)', 'Arbeitszeit (h)', 'Lackierung (h)']
df_auslastung = berechne_auslastung(x_opt, A, b, ressourcen)

print("\n📈 KAPAZITÄTSAUSLASTUNG")
print("=" * 75)
print(df_auslastung.to_string(index=False))

# Visualisierung
fig, ax = plt.subplots(figsize=(10, 5))

x_pos = np.arange(len(ressourcen))
width = 0.35

bars1 = ax.bar(x_pos - width/2, df_auslastung['Verbrauch'], width, 
               label='Verbrauch', color='steelblue')
bars2 = ax.bar(x_pos + width/2, df_auslastung['Kapazität'], width, 
               label='Kapazität', color='lightgray', edgecolor='gray')

# Prozentwerte über Balken
for i, (bar, pct) in enumerate(zip(bars1, df_auslastung['Auslastung (%)'])):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold')

ax.set_ylabel('Einheiten')
ax.set_title('Kapazitätsauslastung bei optimaler Produktion')
ax.set_xticks(x_pos)
ax.set_xticklabels(ressourcen)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Interpretation
print("\n💡 INTERPRETATION:")
for _, row in df_auslastung.iterrows():
    if row['Auslastung (%)'] >= 99.9:
        print(f"   ⚠️  {row['Ressource']}: ENGPASS (100% ausgelastet) - bindende Nebenbedingung")
    elif row['Auslastung (%)'] >= 90:
        print(f"   🟡 {row['Ressource']}: Hohe Auslastung ({row['Auslastung (%)']:.1f}%)")
    else:
        print(f"   🟢 {row['Ressource']}: Reserve vorhanden ({row['Schlupf']:.1f} Einheiten frei)")

---

## 5. Graphische Lösung (2D-Vereinfachung)

Für das Verständnis der linearen Optimierung ist die **graphische Lösung** sehr hilfreich. Da wir nur 2 Dimensionen visualisieren können, betrachten wir eine vereinfachte Version mit nur **Tischen** und **Stühlen**.

### Konzepte der graphischen Lösung:

1. **Zulässiger Bereich**: Der Bereich, in dem alle Nebenbedingungen erfüllt sind (Polygon)
2. **Eckpunkte**: Schnittpunkte der Nebenbedingungen - hier liegt das Optimum!
3. **Isoquanten der Zielfunktion**: Linien gleichen Zielfunktionswertes

### Fundamentalsatz der linearen Optimierung

> Das Optimum eines linearen Programms liegt **immer an einem Eckpunkt** des zulässigen Bereichs (falls eine endliche Lösung existiert).

In [ ]:
# ===================================================================
# Graphische Lösung für 2D-Problem (Tische und Stühle)
# ===================================================================

def plot_2d_optimierung():
    """
    Erstellt eine graphische Darstellung für das 2D-Optimierungsproblem
    (nur Tische und Stühle).
    """
    
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Bereich für x1 (Tische)
    x1 = np.linspace(0, 100, 1000)
    
    # Nebenbedingungen als Funktionen von x1 umgestellt nach x2
    # Holz: 4x₁ + x₂ ≤ 310 => x₂ ≤ 310 - 4x₁
    x2_holz = 310 - 4*x1
    
    # Arbeitszeit: 2x₁ + 1.5x₂ ≤ 260 => x₂ ≤ (260 - 2x₁) / 1.5
    x2_arbeitszeit = (260 - 2*x1) / 1.5
    
    # Lackierung: 3x₁ + x₂ ≤ 250 => x₂ ≤ 250 - 3x₁
    x2_lackierung = 250 - 3*x1
    
    # Nebenbedingungen plotten
    ax.plot(x1, x2_holz, 'r-', linewidth=2.5, label='Holz: 4x₁ + x₂ ≤ 310')
    ax.plot(x1, x2_arbeitszeit, 'b-', linewidth=2.5, label='Arbeitszeit: 2x₁ + 1.5x₂ ≤ 260')
    ax.plot(x1, x2_lackierung, 'g-', linewidth=2.5, label='Lackierung: 3x₁ + x₂ ≤ 250')
    
    # Zulässigen Bereich berechnen und schattieren
    x1_fill = np.linspace(0, 83.33, 500)
    x2_upper = np.minimum(
        np.minimum(310 - 4*x1_fill, (260 - 2*x1_fill) / 1.5),
        250 - 3*x1_fill
    )
    x2_upper = np.maximum(x2_upper, 0)
    
    ax.fill_between(x1_fill, 0, x2_upper, alpha=0.3, color='lightblue', 
                   label='Zulässiger Bereich')
    
    # Eckpunkte berechnen und markieren
    eckpunkte = [
        (0, 0, 'O'),
        (77.5, 0, 'A'),           # Holz-Schnitt mit x₂=0
        (60, 70, 'B'),            # Holz-Lack Schnitt
        (46, 112, 'C'),           # Arbeit-Lack Schnitt (OPTIMUM!)
        (0, 173.33, 'D')          # Arbeitszeit-Schnitt mit x₁=0
    ]
    
    for x, y, label in eckpunkte:
        z_val = 80*x + 40*y
        ax.plot(x, y, 'ko', markersize=8)
        ax.annotate(f'{label}\n({x:.0f}, {y:.0f})\nZ={z_val:.0f}€', 
                   xy=(x, y), xytext=(x+5, y+10),
                   fontsize=9, ha='left')
    
    # Zielfunktions-Isolinien (Isoquanten)
    z_werte = [4000, 6000, 8160]  # 8160 ist das Optimum
    for z_wert in z_werte:
        # Z = 80*x1 + 40*x2 = z_wert => x2 = (z_wert - 80*x1) / 40
        x2_ziel = (z_wert - 80*x1) / 40
        style = '-' if z_wert == 8160 else '--'
        color = 'orange' if z_wert == 8160 else 'gray'
        lw = 2.5 if z_wert == 8160 else 1.5
        ax.plot(x1, x2_ziel, linestyle=style, alpha=0.8, color=color, linewidth=lw,
               label=f'Zielfunktion Z = {z_wert}€' if z_wert in [6000, 8160] else '')
    
    # 2D-Optimierung lösen zur Bestätigung
    c_2d = [-80, -40]
    A_2d = [[4, 1], [2, 1.5], [3, 1]]
    b_2d = [310, 260, 250]
    bounds_2d = [(0, None), (0, None)]
    
    result_2d = linprog(c_2d, A_ub=A_2d, b_ub=b_2d, bounds=bounds_2d, method='highs')
    
    if result_2d.success:
        opt_x1, opt_x2 = result_2d.x
        opt_z = -result_2d.fun
        
        # Optimum markieren
        ax.plot(opt_x1, opt_x2, 'r*', markersize=25, markeredgecolor='black', markeredgewidth=1.5,
               label=f'OPTIMUM: ({opt_x1:.0f}, {opt_x2:.0f}), Z*={opt_z:.0f}€')
    
    # Achsenbeschriftungen und Formatierung
    ax.set_xlim(-5, 100)
    ax.set_ylim(-5, 200)
    ax.set_xlabel('Anzahl Tische (x₁)', fontsize=12)
    ax.set_ylabel('Anzahl Stühle (x₂)', fontsize=12)
    ax.set_title('Graphische Lösung der Linearen Optimierung\n(2D-Vereinfachung: nur Tische und Stühle)', 
                fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_axisbelow(True)
    
    # Achsenlinien hervorheben
    ax.axhline(y=0, color='k', linewidth=1.5)
    ax.axvline(x=0, color='k', linewidth=1.5)
    
    plt.tight_layout()
    plt.show()
    
    return result_2d

print("📈 GRAPHISCHE LÖSUNG (2D-Vereinfachung)")
print("=" * 55)
result_2d = plot_2d_optimierung()

print("\n💡 ERKENNTNISSE:")
print("   1. Das Optimum (Punkt C) liegt am SCHNITTPUNKT von Arbeitszeit")
print("      und Lackierung → beide Ressourcen sind Engpässe!")
print("   2. Die Holz-Beschränkung ist NICHT bindend (Schlupf vorhanden)")
print("   3. Die optimale Zielfunktionslinie (orange) berührt den")
print("      zulässigen Bereich genau im Eckpunkt C.")

---

## 6. Sensitivitätsanalyse

Die **Sensitivitätsanalyse** untersucht, wie sich die optimale Lösung ändert, wenn sich die Parameter des Problems ändern.

### Wichtige Konzepte:

| Begriff | Definition | Interpretation |
|---------|-----------|----------------|
| **Schattenpreis** | Änderung des Zielfunktionswertes bei Erhöhung einer Kapazität um 1 Einheit | Wert einer zusätzlichen Ressourceneinheit |
| **Bindende NB** | Nebenbedingung mit Schlupf = 0 | Ressource ist Engpass |
| **Nicht-bindende NB** | Nebenbedingung mit Schlupf > 0 | Ressource hat Überschuss |
| **Stabilitätsbereich** | Bereich, in dem die Basislösung optimal bleibt | Gültigkeitsbereich der Analyse |

In [ ]:
# ===================================================================
# Sensitivitätsanalyse
# ===================================================================

def sensitivitaetsanalyse(ressource_idx, ressource_name, c, A, b, variation_prozent=30, n_punkte=31):
    """
    Führt eine Sensitivitätsanalyse für eine Ressourcenkapazität durch.
    
    Parameter:
    ----------
    ressource_idx : int
        Index der Ressource (0=Holz, 1=Arbeitszeit, 2=Lackierung)
    ressource_name : str
        Name der Ressource für die Beschriftung
    variation_prozent : float
        Prozentuale Variation nach oben und unten
    n_punkte : int
        Anzahl der Datenpunkte
    """
    
    basis_kapazitaet = b[ressource_idx]
    variationen = np.linspace(
        basis_kapazitaet * (1 - variation_prozent/100),
        basis_kapazitaet * (1 + variation_prozent/100), 
        n_punkte
    )
    
    # Basis-Zielfunktionswert berechnen
    result_basis = linprog(-c, A_ub=A, b_ub=b, bounds=[(0, None)]*3, method='highs')
    z_basis = -result_basis.fun
    
    deckungsbeitraege = []
    
    for neue_kapazitaet in variationen:
        # Neue rechte Seite erstellen
        b_neu = b.copy()
        b_neu[ressource_idx] = neue_kapazitaet
        
        # Optimierung
        result = linprog(-c, A_ub=A, b_ub=b_neu, bounds=[(0, None)]*3, method='highs')
        
        if result.success:
            deckungsbeitraege.append(-result.fun)
        else:
            deckungsbeitraege.append(np.nan)
    
    # Schattenpreis berechnen (numerische Ableitung am Basispunkt)
    idx_basis = len(variationen) // 2
    delta = variationen[1] - variationen[0]
    schattenpreis = (deckungsbeitraege[idx_basis + 1] - deckungsbeitraege[idx_basis - 1]) / (2 * delta)
    
    # Visualisierung
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Zielfunktionswert über Kapazität
    ax1.plot(variationen, deckungsbeitraege, 'b-', linewidth=2)
    ax1.axvline(x=basis_kapazitaet, color='r', linestyle='--', linewidth=2,
               label=f'Basis: {basis_kapazitaet}')
    ax1.axhline(y=z_basis, color='g', linestyle=':', alpha=0.7,
               label=f'Aktuelles Optimum: {z_basis:.0f}€')
    ax1.fill_between(variationen, min(deckungsbeitraege)*0.98, deckungsbeitraege, 
                    alpha=0.2, color='blue')
    ax1.set_xlabel(f'{ressource_name} Kapazität', fontsize=11)
    ax1.set_ylabel('Maximaler Deckungsbeitrag (€)', fontsize=11)
    ax1.set_title(f'Sensitivität bezüglich {ressource_name}', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Schattenpreis annotieren
    if abs(schattenpreis) > 0.1:
        ax1.annotate(f'Schattenpreis:\n≈ {schattenpreis:.2f} €/Einheit',
                    xy=(basis_kapazitaet, z_basis),
                    xytext=(basis_kapazitaet * 1.1, z_basis * 0.95),
                    fontsize=11, fontweight='bold',
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8),
                    arrowprops=dict(arrowstyle='->', color='black'))
    
    # Plot 2: Änderung in Prozent
    prozent_aenderung = [(db / z_basis - 1) * 100 for db in deckungsbeitraege]
    kapazitaet_prozent = [(k / basis_kapazitaet - 1) * 100 for k in variationen]
    
    ax2.plot(kapazitaet_prozent, prozent_aenderung, 'g-', linewidth=2)
    ax2.axhline(y=0, color='k', linestyle='-', linewidth=1)
    ax2.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Basisfall')
    ax2.fill_between(kapazitaet_prozent, 0, prozent_aenderung, 
                    where=[p > 0 for p in prozent_aenderung], alpha=0.3, color='green')
    ax2.fill_between(kapazitaet_prozent, 0, prozent_aenderung, 
                    where=[p < 0 for p in prozent_aenderung], alpha=0.3, color='red')
    ax2.set_xlabel('Kapazitätsänderung (%)', fontsize=11)
    ax2.set_ylabel('Änderung Deckungsbeitrag (%)', fontsize=11)
    ax2.set_title('Relative Sensitivität', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return schattenpreis

print("🔍 SENSITIVITÄTSANALYSE")
print("=" * 50)

# Analyse für alle drei Ressourcen
schattenpreise = {}
ressourcen_namen = ['Holz (m²)', 'Arbeitszeit (h)', 'Lackierung (h)']

for idx, name in enumerate(ressourcen_namen):
    print(f"\n▶ Analyse: {name}")
    sp = sensitivitaetsanalyse(idx, name, c, A, b)
    schattenpreise[name] = sp
    print(f"   Schattenpreis: {sp:.2f} €/Einheit")

In [ ]:
# ===================================================================
# Zusammenfassung Schattenpreise
# ===================================================================

print("\n📊 ZUSAMMENFASSUNG SCHATTENPREISE")
print("=" * 70)

df_schattenpreise = pd.DataFrame({
    'Ressource': list(schattenpreise.keys()),
    'Schattenpreis (€/Einheit)': [f"{sp:.2f}" for sp in schattenpreise.values()],
    'Bindend?': [
        'JA (Engpass)' if sp > 0.1 else 'NEIN (Reserve)'
        for sp in schattenpreise.values()
    ]
})

print(df_schattenpreise.to_string(index=False))

# Identifiziere den wertvollsten Engpass
max_sp_ressource = max(schattenpreise, key=schattenpreise.get)
max_sp = schattenpreise[max_sp_ressource]

print(f"\n💡 MANAGEMENT-EMPFEHLUNG:")
if max_sp > 0:
    print(f"   Die wertvollste Kapazitätserweiterung wäre bei '{max_sp_ressource}'.")
    print(f"   Jede zusätzliche Einheit bringt ca. {max_sp:.2f}€ mehr Deckungsbeitrag.")
    print(f"\n   Investitionen in Kapazitäten mit Schattenpreis = 0 sind NICHT sinnvoll,")
    print(f"   da dort bereits Überkapazitäten bestehen.")
else:
    print(f"   Alle Ressourcen haben Überkapazitäten. Keine Erweiterung nötig.")

---

## 7. Erweiterte Fallstudie: Smartphone-Fertigung

Um die Konzepte zu vertiefen, betrachten wir ein komplexeres Beispiel aus der **Elektronikfertigung**.

### Szenario

Ein Smartphone-Hersteller produziert 5 verschiedene Modelle. Zusätzlich zu den Kapazitätsbeschränkungen gibt es eine **Mindestproduktionsanforderung** für Premium-Modelle (Marketingvorgabe).

In [ ]:
# ===================================================================
# Fallstudie: Smartphone-Produktion
# ===================================================================

print("📱 FALLSTUDIE: Smartphone-Produktion")
print("=" * 65)

# Produktdaten - so gewählt, dass günstige Modelle effizienter sind
smartphones = {
    'Basic':    {'DB_EUR': 40,  'Montage_min': 8,  'Test_min': 3,  'Verpackung_min': 2},
    'Standard': {'DB_EUR': 65,  'Montage_min': 12, 'Test_min': 5,  'Verpackung_min': 3},
    'Premium':  {'DB_EUR': 90,  'Montage_min': 20, 'Test_min': 10, 'Verpackung_min': 5},
    'Pro':      {'DB_EUR': 110, 'Montage_min': 30, 'Test_min': 15, 'Verpackung_min': 8},
    'Ultra':    {'DB_EUR': 130, 'Montage_min': 45, 'Test_min': 25, 'Verpackung_min': 12}
}

# Kapazitäten (pro Schicht = 8 Stunden)
smartphone_kapazitaeten = {
    'Montage_min': 480,    # 8 Stunden = 480 Minuten
    'Test_min': 240,       # 4 Stunden = 240 Minuten
    'Verpackung_min': 150  # 2.5 Stunden = 150 Minuten
}

# Zusätzliche Anforderung: Mindestens 10 High-End-Modelle (Pro + Ultra)
min_highend = 10

# Daten anzeigen
df_smartphones = pd.DataFrame(smartphones).T
df_smartphones.index.name = 'Modell'
print("\nProduktdaten:")
print(df_smartphones.to_string())

print(f"\nKapazitäten (pro Schicht):")
for k, v in smartphone_kapazitaeten.items():
    print(f"   {k}: {v} Minuten ({v/60:.1f} Stunden)")

print(f"\n⚠️  Marketing-Vorgabe: Mindestens {min_highend} High-End-Modelle (Pro + Ultra)")

# Effizienz berechnen
print("\n📊 EFFIZIENZ (DB pro Verpackungsminute - oft der Engpass):")
print("-" * 50)
for modell, info in smartphones.items():
    eff = info['DB_EUR'] / info['Verpackung_min']
    print(f"   {modell:10s}: {eff:.2f} €/min")
print("\n💡 Basic und Standard sind am effizientesten!")

In [ ]:
# ===================================================================
# Lösung Smartphone-Problem
# ===================================================================

def solve_smartphone_problem(mit_mindestanforderung=True):
    """
    Löst das Smartphone-Produktionsproblem.
    
    Parameter:
    ----------
    mit_mindestanforderung : bool
        Ob die Mindestanforderung für High-End-Modelle berücksichtigt werden soll
    """
    
    # Zielfunktion (negativ für Maximierung)
    c_sp = [-40, -65, -90, -110, -130]
    
    # Ungleichungs-Nebenbedingungen (Kapazitäten)
    A_ub_sp = [
        [8, 12, 20, 30, 45],   # Montage
        [3, 5, 10, 15, 25],    # Test
        [2, 3, 5, 8, 12]       # Verpackung
    ]
    b_ub_sp = [480, 240, 150]
    
    # Grenzen
    bounds_sp = [(0, None)] * 5
    
    if mit_mindestanforderung:
        # Mindestanforderung: Pro + Ultra >= 10
        # Umformuliert als Ungleichung: -Pro - Ultra <= -10
        A_ub_sp.append([0, 0, 0, -1, -1])
        b_ub_sp.append(-min_highend)
    
    result = linprog(c_sp, A_ub=A_ub_sp, b_ub=b_ub_sp, bounds=bounds_sp, method='highs')
    
    return result

# Lösung ohne Mindestanforderung
print("\n" + "="*60)
print("🔹 Lösung OHNE Mindestanforderung (rein ökonomisch optimal):")
print("="*60)
result_ohne = solve_smartphone_problem(mit_mindestanforderung=False)

if result_ohne.success:
    modelle = list(smartphones.keys())
    print(f"\n{'Modell':<12} {'Menge':>8} {'DB gesamt':>12}")
    print("-" * 35)
    total_db = 0
    for i, modell in enumerate(modelle):
        menge = result_ohne.x[i]
        db = menge * smartphones[modell]['DB_EUR']
        total_db += db
        if menge > 0.01:
            print(f"{modell:<12} {menge:>8.1f} {db:>12.2f} €")
    print("-" * 35)
    print(f"{'GESAMT':<12} {sum(result_ohne.x):>8.1f} {-result_ohne.fun:>12.2f} €")
    
    highend_ohne = result_ohne.x[3] + result_ohne.x[4]
    print(f"\n   High-End produziert: {highend_ohne:.1f} Stück")

# Lösung mit Mindestanforderung
print("\n" + "="*60)
print(f"🔹 Lösung MIT Mindestanforderung (≥{min_highend} High-End):")
print("="*60)
result_mit = solve_smartphone_problem(mit_mindestanforderung=True)

if result_mit.success:
    print(f"\n{'Modell':<12} {'Menge':>8} {'DB gesamt':>12}")
    print("-" * 35)
    for i, modell in enumerate(modelle):
        menge = result_mit.x[i]
        db = menge * smartphones[modell]['DB_EUR']
        if menge > 0.01:
            print(f"{modell:<12} {menge:>8.1f} {db:>12.2f} €")
    print("-" * 35)
    print(f"{'GESAMT':<12} {sum(result_mit.x):>8.1f} {-result_mit.fun:>12.2f} €")
    
    highend_mit = result_mit.x[3] + result_mit.x[4]
    print(f"\n   High-End produziert: {highend_mit:.1f} Stück")
    
    # Kosten der Zusatzanforderung
    kosten_zusatzanforderung = -result_ohne.fun - (-result_mit.fun)
    print(f"\n💰 KOSTEN DER MARKETING-VORGABE:")
    print(f"   Entgangener Deckungsbeitrag: {kosten_zusatzanforderung:,.2f} €")
    print(f"   Das entspricht {kosten_zusatzanforderung/(-result_ohne.fun)*100:.1f}% des maximalen DB")

---

## 8. Aufgaben für Studierende

Bearbeiten Sie die folgenden Aufgaben, um Ihr Verständnis der linearen Optimierung zu vertiefen.

### ✏️ Aufgabe 1: Kapazitätserweiterung

Die WoodCraft GmbH kann **eine** ihrer Kapazitäten um **20%** erweitern. 

**Fragen:**
1. Welche Kapazität sollte erweitert werden, um den größten Nutzen zu erzielen?
2. Um wie viel Euro steigt der maximale Deckungsbeitrag?
3. Welche Kapazitätserweiterung würde sich NICHT lohnen und warum?

*Hinweis: Nutzen Sie die Schattenpreise aus der Sensitivitätsanalyse!*

In [ ]:
# Ihre Lösung für Aufgabe 1:
# ===========================

# TODO: Identifizieren Sie die Kapazität mit dem höchsten Schattenpreis
# TODO: Berechnen Sie die neue Kapazität (+ 20%)
# TODO: Lösen Sie das neue Problem und vergleichen Sie die Ergebnisse



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 1 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 1: Kapazitätserweiterung")
print("=" * 65)

# Schritt 1: Schattenpreise analysieren
print("\nSchritt 1: Analyse der Schattenpreise")
print("-" * 45)
for name, sp in schattenpreise.items():
    status = "ENGPASS" if sp > 0.1 else "Überkapazität"
    print(f"   {name}: {sp:.2f} €/Einheit  [{status}]")

beste_ressource = max(schattenpreise, key=schattenpreise.get)
print(f"\n→ Höchster Schattenpreis: {beste_ressource}")

# Schritt 2: Neue Kapazität berechnen
print("\nSchritt 2: Kapazitätserweiterung um 20%")
print("-" * 45)

ressourcen_liste = ['Holz (m²)', 'Arbeitszeit (h)', 'Lackierung (h)']
idx_beste = ressourcen_liste.index(beste_ressource)
alte_kapazitaet = b[idx_beste]
neue_kapazitaet = alte_kapazitaet * 1.20

print(f"   Alte Kapazität {beste_ressource}: {alte_kapazitaet}")
print(f"   Neue Kapazität {beste_ressource}: {neue_kapazitaet} (+20%)")

# Schritt 3: Optimierung mit neuer Kapazität
print("\nSchritt 3: Neue Optimierung durchführen")
print("-" * 45)

b_neu = b.copy()
b_neu[idx_beste] = neue_kapazitaet

result_neu = linprog(-c, A_ub=A, b_ub=b_neu, bounds=[(0, None)]*3, method='highs')

if result_neu.success:
    z_neu = -result_neu.fun
    verbesserung = z_neu - z_opt
    
    print(f"   Alter max. DB: {z_opt:,.2f} €")
    print(f"   Neuer max. DB: {z_neu:,.2f} €")
    print(f"   ─────────────────────────────")
    print(f"   Verbesserung:  {verbesserung:,.2f} € (+{verbesserung/z_opt*100:.1f}%)")
    
    print("\n📊 Neue optimale Produktionsmengen:")
    for i, (prod, menge_neu, menge_alt) in enumerate(zip(['Tische', 'Stühle', 'Schränke'], result_neu.x, x_opt)):
        diff = menge_neu - menge_alt
        print(f"   {prod}: {menge_neu:.1f} (vorher: {menge_alt:.1f}, Δ: {diff:+.1f})")

print("\n" + "-" * 45)
print("Antwort auf Frage 3:")
for name, sp in schattenpreise.items():
    if sp < 0.1:
        print(f"   Eine Erweiterung von '{name}' würde sich NICHT lohnen,")
        print(f"   da der Schattenpreis ≈ 0 ist (keine Engpass-Ressource).")

### ✏️ Aufgabe 2: Neues Produkt einführen

Die WoodCraft GmbH möchte ein neues Produkt **"Regal"** einführen:

| Parameter | Wert |
|-----------|------|
| Deckungsbeitrag | 55 € |
| Holzbedarf | 1.5 m² |
| Arbeitszeit | 1.2 h |
| Lackierung | 1.5 h |

**Fragen:**
1. Wie ändert sich das optimale Produktionsprogramm?
2. Sollte das Regal produziert werden? Warum (nicht)?
3. Wie viel zusätzlicher Deckungsbeitrag wird erzielt?

In [ ]:
# Ihre Lösung für Aufgabe 2:
# ===========================

# TODO: Erweitern Sie das Modell um das neue Produkt "Regal"
# TODO: Lösen Sie das erweiterte Problem
# TODO: Analysieren Sie, ob das Regal produziert werden sollte



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 2 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 2: Neues Produkt 'Regal'")
print("=" * 65)

# Erweitertes Modell mit 4 Produkten
c_erweitert = np.array([80, 40, 90, 55])  # Tisch, Stuhl, Schrank, Regal

A_erweitert = np.array([
    [4.0, 1.0, 3.0, 1.5],  # Holz
    [2.0, 1.5, 3.0, 1.2],  # Arbeitszeit
    [3.0, 1.0, 2.0, 1.5]   # Lackierung
])

print("\nNeue Produktmatrix (inkl. Regal):")
print("         Tisch  Stuhl  Schrank  Regal")
print(f"Holz:    {A_erweitert[0]}")
print(f"Arbeit:  {A_erweitert[1]}")
print(f"Lack:    {A_erweitert[2]}")
print(f"DB:      {c_erweitert}")

# Effizienzvergleich
print("\n📊 Effizienzvergleich (DB pro Ressourceneinheit):")
print("-" * 55)
print(f"{'Produkt':<10} {'DB/Holz':>10} {'DB/Arbeit':>12} {'DB/Lackierung':>14}")
for i, prod in enumerate(['Tisch', 'Stuhl', 'Schrank', 'Regal']):
    db_h = c_erweitert[i] / A_erweitert[0, i]
    db_a = c_erweitert[i] / A_erweitert[1, i]
    db_l = c_erweitert[i] / A_erweitert[2, i]
    print(f"{prod:<10} {db_h:>10.2f} {db_a:>12.2f} {db_l:>14.2f}")

print("\n💡 Das Regal hat den höchsten DB pro Arbeitsstunde (45.83 €/h)!")

# Optimierung
b_erweitert = b  # Kapazitäten bleiben gleich

result_erweitert = linprog(
    -c_erweitert, 
    A_ub=A_erweitert, 
    b_ub=b_erweitert, 
    bounds=[(0, None)]*4, 
    method='highs'
)

if result_erweitert.success:
    x_erweitert = result_erweitert.x
    z_erweitert = -result_erweitert.fun
    
    print("\n" + "="*55)
    print("ERGEBNIS: Optimale Produktionsmengen MIT Regal:")
    print("="*55)
    produkte_erweitert = ['Tische', 'Stühle', 'Schränke', 'Regale']
    print(f"\n{'Produkt':<12} {'Menge':>8} {'DB gesamt':>12}")
    print("-" * 35)
    for prod, menge, db in zip(produkte_erweitert, x_erweitert, c_erweitert):
        if menge > 0.01:
            print(f"{prod:<12} {menge:>8.2f} {menge*db:>12.2f} €")
    print("-" * 35)
    print(f"{'GESAMT':>12} {sum(x_erweitert):>8.2f} {z_erweitert:>12.2f} €")
    
    print("\n📈 Vergleich mit Situation ohne Regal:")
    print(f"   Ohne Regal: {z_opt:,.2f} €")
    print(f"   Mit Regal:  {z_erweitert:,.2f} €")
    print(f"   Differenz:  {z_erweitert - z_opt:,.2f} € (+{(z_erweitert/z_opt-1)*100:.1f}%)")
    
    if x_erweitert[3] > 0.01:
        print("\n✅ EMPFEHLUNG: Das Regal sollte ins Produktionsprogramm aufgenommen werden!")
        print(f"   Es werden {x_erweitert[3]:.1f} Regale produziert.")
    else:
        print("\n❌ Das Regal wird NICHT produziert (nicht profitabel genug).")

### ✏️ Aufgabe 3: Mindestproduktion

Aufgrund eines Kundenvertrags müssen **mindestens 120 Stühle** pro Woche produziert werden.

**Fragen:**
1. Wie wirkt sich diese Beschränkung auf die optimale Lösung aus?
2. Ist die Beschränkung bindend (d.h. werden ohne sie weniger Stühle produziert)?
3. Was kostet diese Anforderung das Unternehmen (in Form von entgangenem Deckungsbeitrag)?

---

### Exkurs: Formulierung von ≥-Beschränkungen in SciPy

**Problem:** SciPy's `linprog` akzeptiert Ungleichungen **nur in der Form** $A \cdot x \leq b$.

**Lösung:** Es gibt zwei Möglichkeiten:

| Methode | Beschreibung | Anwendung |
|---------|--------------|----------|
| **1. bounds** | Untergrenze direkt setzen | Einfache Variablenbeschränkungen wie $x_j \geq L$ |
| **2. Umformung** | $x \geq L$ wird zu $-x \leq -L$ | Komplexere Beschränkungen wie $\sum a_j x_j \geq b$ |

**Beispiel für $x_2 \geq 120$:**

```python
# Methode 1: bounds (EMPFOHLEN für einfache Untergrenzen)
bounds = [(0, None), (120, None), (0, None)]  # x₂ ≥ 120

# Methode 2: Umformung (für komplexere Beschränkungen)
# x₂ ≥ 120  →  -x₂ ≤ -120
A_zusatz = [0, -1, 0]  # Koeffizient -1 für x₂
b_zusatz = -120
```

In [ ]:
# Ihre Lösung für Aufgabe 3:
# ===========================

# TODO: Fügen Sie die Mindestproduktionsbeschränkung hinzu (x₂ >= 120)
# TODO: Nutzen Sie entweder bounds ODER die Umformungsmethode
# TODO: Vergleichen Sie mit der Lösung ohne Mindestproduktion
# TODO: Berechnen Sie die "Kosten" der Beschränkung



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 3 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 3: Mindestproduktion von 120 Stühlen")
print("=" * 70)

min_stuehle = 120

# Prüfen: Wie viele Stühle werden ohne Beschränkung produziert?
print(f"\nAktuelle optimale Lösung (ohne Mindestmenge):")
print(f"   Tische:   {x_opt[0]:.2f} Stück")
print(f"   Stühle:   {x_opt[1]:.2f} Stück")
print(f"   Schränke: {x_opt[2]:.2f} Stück")
print(f"   Max. DB:  {z_opt:.2f} €")

if x_opt[1] >= min_stuehle:
    print(f"\n✅ Die Mindestmenge von {min_stuehle} Stühlen wird bereits erfüllt!")
    print(f"   Die Beschränkung ist NICHT bindend.")
else:
    print(f"\n⚠️  Es werden nur {x_opt[1]:.2f} Stühle produziert.")
    print(f"   Die Mindestmenge von {min_stuehle} erfordert eine Anpassung!")

# =====================================================
# METHODE 1: Über bounds-Parameter (EMPFOHLEN)
# =====================================================
print("\n" + "="*70)
print("METHODE 1: Über bounds-Parameter (empfohlen für einfache Untergrenzen)")
print("="*70)

print("\nCode:")
print('   bounds = [(0, None), (120, None), (0, None)]  # x₂ ≥ 120')

bounds_mit_minimum = [
    (0, None),           # x₁ (Tische): ≥ 0
    (min_stuehle, None), # x₂ (Stühle): ≥ 120  <-- Mindestmenge!
    (0, None)            # x₃ (Schränke): ≥ 0
]

result_methode1 = linprog(
    -c, 
    A_ub=A, 
    b_ub=b, 
    bounds=bounds_mit_minimum, 
    method='highs'
)

if result_methode1.success:
    print(f"\nErgebnis Methode 1:")
    print(f"   Tische:   {result_methode1.x[0]:.2f}")
    print(f"   Stühle:   {result_methode1.x[1]:.2f}")
    print(f"   Schränke: {result_methode1.x[2]:.2f}")
    print(f"   Max. DB:  {-result_methode1.fun:.2f} €")

# =====================================================
# METHODE 2: Über zusätzliche Nebenbedingung (Umformung)
# =====================================================
print("\n" + "="*70)
print("METHODE 2: Über zusätzliche Nebenbedingung (für komplexere Fälle)")
print("="*70)

print("\nMathematische Umformung:")
print(f"   x₂ ≥ {min_stuehle}")
print(f"   ↓ (beide Seiten mit -1 multiplizieren, Ungleichung dreht sich um)")
print(f"   -x₂ ≤ -{min_stuehle}")
print("\n   linprog akzeptiert nur ≤-Ungleichungen, daher diese Umformung!")

print("\nCode:")
print('   A_erweitert = np.vstack([A, [0, -1, 0]])  # Zeile für -x₂')
print('   b_erweitert = np.append(b, -120)          # rechte Seite -120')

A_mit_minimum = np.vstack([A, [0, -1, 0]])  # Zeile [0, -1, 0] für -x₂
b_mit_minimum = np.append(b, -min_stuehle)   # -120

result_methode2 = linprog(
    -c, 
    A_ub=A_mit_minimum, 
    b_ub=b_mit_minimum, 
    bounds=[(0, None)]*3, 
    method='highs'
)

if result_methode2.success:
    print(f"\nErgebnis Methode 2:")
    print(f"   Tische:   {result_methode2.x[0]:.2f}")
    print(f"   Stühle:   {result_methode2.x[1]:.2f}")
    print(f"   Schränke: {result_methode2.x[2]:.2f}")
    print(f"   Max. DB:  {-result_methode2.fun:.2f} €")

# =====================================================
# Vergleich und Kostenanalyse
# =====================================================
print("\n" + "="*70)
print("VERGLEICH UND KOSTENANALYSE")
print("="*70)

if result_methode1.success and result_methode2.success:
    # Prüfen ob beide Methoden gleiches Ergebnis liefern
    diff = np.abs(result_methode1.x - result_methode2.x).sum()
    if diff < 0.01:
        print("\n✅ Beide Methoden liefern das gleiche Ergebnis!")
    
    z_min = -result_methode1.fun
    x_min = result_methode1.x
    
    print(f"\n{'':12s} {'Ohne Minimum':>14} {'Mit Minimum':>14} {'Differenz':>12}")
    print("-" * 55)
    for i, prod in enumerate(['Tische', 'Stühle', 'Schränke']):
        diff = x_min[i] - x_opt[i]
        print(f"{prod:<12} {x_opt[i]:>14.2f} {x_min[i]:>14.2f} {diff:>+12.2f}")
    print("-" * 55)
    print(f"{'Max. DB (€)':<12} {z_opt:>14.2f} {z_min:>14.2f} {z_min-z_opt:>+12.2f}")
    
    kosten_minimum = z_opt - z_min
    
    print(f"\n💰 KOSTEN DER MINDESTPRODUKTIONSANFORDERUNG:")
    print(f"   Entgangener Deckungsbeitrag: {kosten_minimum:,.2f} €")
    print(f"   Prozentuale Reduktion:       {kosten_minimum/z_opt*100:.2f}%")
    
    if kosten_minimum > 0:
        print(f"\n⚠️  FAZIT:")
        print(f"   Die Mindestproduktion von {min_stuehle} Stühlen kostet")
        print(f"   {kosten_minimum:,.2f}€ pro Woche an entgangenem Gewinn!")
        print(f"\n   Der Kundenvertrag sollte mindestens {kosten_minimum:,.2f}€")
        print(f"   Mehrwert pro Woche bringen, damit er sich lohnt.")
    else:
        print(f"\n✅ Die Mindestanforderung verursacht KEINE zusätzlichen Kosten.")
else:
    print("\n❌ Problem ist nicht lösbar!")
    print(f"   Die Mindestanforderung von {min_stuehle} Stühlen kann mit den")
    print(f"   verfügbaren Kapazitäten nicht erfüllt werden.")

---

## 9. Zusammenfassung und Ausblick

### Was haben wir gelernt?

| Thema | Kernaussage |
|-------|-------------|
| **Modellierung** | Produktionsplanungsprobleme lassen sich als LP-Probleme mit Zielfunktion und Nebenbedingungen formulieren |
| **Lösung** | Python/SciPy ermöglicht effiziente Lösung auch komplexer Probleme (Methode: `linprog`) |
| **Graphische Methode** | Das Optimum liegt immer an einem Eckpunkt des zulässigen Bereichs |
| **Sensitivität** | Schattenpreise zeigen den Wert zusätzlicher Ressourcen; nur Engpassressourcen haben positive Schattenpreise |
| **Zusatzbedingungen** | Marketing- oder Vertragsvorgaben können den optimalen DB reduzieren |

### Formulierung verschiedener Beschränkungstypen in SciPy

| Beschränkung | Mathematisch | SciPy-Umsetzung |
|--------------|--------------|----------------|
| Obergrenze Variable | $x_j \leq U$ | `bounds=[(0, U), ...]` |
| Untergrenze Variable | $x_j \geq L$ | `bounds=[(L, None), ...]` |
| Kleiner-gleich | $\sum a_j x_j \leq b$ | `A_ub`, `b_ub` (direkt) |
| Größer-gleich | $\sum a_j x_j \geq b$ | Umformen: $-\sum a_j x_j \leq -b$ |
| Gleichung | $\sum a_j x_j = b$ | `A_eq`, `b_eq` |

### Grenzen der linearen Optimierung

- ❌ Alle Zusammenhänge müssen **linear** sein
- ❌ Variablen sind **kontinuierlich** (keine ganzzahligen Werte garantiert)
- ❌ **Unsicherheiten** werden nicht berücksichtigt
- ❌ **Dynamische Aspekte** (Zeit) nicht abgebildet

### Weiterführende Themen

- **Ganzzahlige Optimierung** (Integer Programming) - wenn nur ganze Stückzahlen erlaubt sind
- **Nichtlineare Optimierung** - bei nichtlinearen Kostenverläufen
- **Stochastische Optimierung** - bei unsicheren Parametern
- **Mehrzieloptimierung** - bei mehreren, konkurrierenden Zielen

In [ ]:
# ===================================================================
# Abschluss und Checkliste
# ===================================================================

print("\n" + "="*70)
print("📋 CHECKLISTE - Lineare Optimierung")
print("="*70)

checkliste = [
    ("LP-Problem formulieren", 
     "Entscheidungsvariablen, Zielfunktion, Nebenbedingungen definieren"),
    ("SciPy linprog anwenden", 
     "Maximierung durch Negation; A_ub/b_ub für ≤; bounds für Variablengrenzen"),
    ("≥-Beschränkungen umformen", 
     "Entweder bounds nutzen oder: x≥L wird zu -x≤-L"),
    ("Graphische Lösung interpretieren", 
     "Zulässiger Bereich als Polygon; Optimum am Eckpunkt"),
    ("Kapazitätsauslastung analysieren", 
     "Bindende vs. nicht-bindende Nebenbedingungen; Schlupfvariablen"),
    ("Sensitivitätsanalyse durchführen", 
     "Schattenpreise berechnen; Engpassressourcen identifizieren"),
    ("Ergebnisse betriebswirtschaftlich deuten", 
     "Produktmix optimieren; Kosten von Zusatzanforderungen bewerten")
]

for i, (punkt, beschreibung) in enumerate(checkliste, 1):
    print(f"\n☐ {i}. {punkt}")
    print(f"   → {beschreibung}")

print("\n" + "="*70)
print("🎓 Viel Erfolg bei der weiteren Vertiefung!")
print("   Nächstes Thema: Warteschlangentheorie")
print("="*70)